In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
EMBEDDING_SIZE = 300

In [ ]:
class ImageEmbedder(nn.Module):
    
    def __init__(self):
        
        
        
    def forward(self, inputs):

In [ ]:
class LSTMCaptioning(nn.Module):
    
    def __init__(self):
        
        
    def forward(self, inputs):
        